In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

import torch.nn as nn
import torch.nn.functional as F


In [2]:
#Refer to Kuangliu's pytorch cifar examples https://github.com/kuangliu/pytorch-cifar/blob/master/models/resnet.py 
#Refer to Pytorch Vision Opensource https://github.com/pytorch/vision/blob/master/torchvision/models/resnet.py
#Refer to Deep Residual Learning for Image Recognition, https://arxiv.org/pdf/1512.03385.pdf
#Refer to Writer(Kaiming He)'s ResNet(caffe) https://github.com/KaimingHe/deep-residual-networks

class BasicBlock(nn.Module):
    expansion = 1
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1, padding=1):
        super(BasicBlock, self).__init__()
        #outtmp = out_channels/self.expansion
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=padding,bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        
        self.conv2 = nn.Conv2d(out_channels, out_channels*self.expansion, kernel_size=kernel_size, stride=1,padding=padding,bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels*self.expansion)
        
        self.shortcut = nn.Sequential()
        if(stride != 1 or in_channels != out_channels*self.expansion):
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels*self.expansion, kernel_size=1, stride=stride,bias=False),
                nn.BatchNorm2d(out_channels*self.expansion)
            )
        
        
    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = F.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        out = out + self.shortcut(x)
        
        out = F.relu(out)
        
        return out
        
        
    
class BottleNeck(nn.Module):
    expansion = 4
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1, padding=1):
        super(BottleNeck, self).__init__()
        #outtmp = out_channels/self.expansion
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, padding=0,bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=kernel_size, stride=stride,padding=padding,bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        
        self.conv3 = nn.Conv2d(out_channels, out_channels*self.expansion, kernel_size=1, stride=1,padding=0,bias=False)
        self.bn3 = nn.BatchNorm2d(out_channels*self.expansion)
        
        
        
        self.shortcut = nn.Sequential()
        if(stride != 1 or in_channels != out_channels*self.expansion):
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels*self.expansion, kernel_size=1, stride=stride,bias=False),
                nn.BatchNorm2d(out_channels*self.expansion)
            )
    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        
        out = F.relu(out)
        
        return out
            
class ResNet(nn.Module):
    last_out = 64
    def __init__(self, block, layers, classes=1000):
        super(ResNet, self).__init__()
        #3x244x244
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3)
        self.bn1 = nn.BatchNorm2d(64)
        #64x122x122
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        #64x56x56
        self.conv2_x = self._make_layer(block, 64, layers[0], stride=1)
        #64*expx56x56
        self.conv3_x = self._make_layer(block, 128, layers[1], stride=2)
        #128*expx28x28
        self.conv4_x = self._make_layer(block, 256, layers[2], stride=2)
        #256*expx14x14
        self.conv5_x = self._make_layer(block, 512, layers[3], stride=2)
        #512*expx7x7
        
        self.avgpool = nn.AvgPool2d(7)
        self.fc = nn.Linear(512*block.expansion, classes)
        
    def _make_layer(self, block, out_channels, num_res, stride=2):
        layers = []
        layers.append(block(self.last_out, out_channels, stride=stride))
        self.last_out = out_channels*block.expansion
        
        for _ in range(1, num_res):
            layers.append(block(self.last_out, out_channels))
            #self.last_out = out_channels
            
        return nn.Sequential(*layers)
    
    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = F.relu(out)
        
        out = self.maxpool(out)
        out = self.conv2_x(out)  
        out = self.conv3_x(out)
        out = self.conv4_x(out)
        f = self.conv5_x(out)
        
        out = self.avgpool(f)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        
        return out, f

In [3]:
def ResNet18():
    return ResNet(BasicBlock, [2,2,2,2],classes=10)

def ResNet34():
    return ResNet(BasicBlock, [3,4,6,3],classes=10)

def ResNet50():
    return ResNet(BottleNeck, [3,4,6,3],classes=10)

def ResNet101():
    return ResNet(BottleNeck, [3,4,23,3],classes=10)

def ResNet152():
    return ResNet(BottleNeck, [3,8,36,3],classes=10)

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [5]:
print(device)

cuda:0


In [6]:
net = ResNet152()
net = net.to(device)
param = list(net.parameters())

In [30]:
transform_train = transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.46666667, 0.875), ratio=(1,1)),
    #The image is resized with its shorter side randomly sampled in [256, 480] for scale augmentation
    transforms.RandomHorizontalFlip(),
    #is randomly sampled from an image or its horizontal flip
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    #the per-pixel mean subtracted(좀 더 간편한 노멀라이즈 방법으로 대체)
])

transform_test = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    #the per-pixel mean subtracted(좀 더 간편한 노멀라이즈 방법으로 대체)
])

trainset = torchvision.datasets.STL10(root='./data', split='train', download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True)

testset = torchvision.datasets.STL10(root='./data', split='test', download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=32, shuffle=False)

classes =  ['airplance', 'bird', 'car', 'cat', 'deer', 'dog', 'horse', 'monkey', 'ship', 'truck']

import torch.optim as optim


Files already downloaded and verified
Files already downloaded and verified


In [11]:
def train(net, optimizer, criterion, printby=50):
    running_loss = 0.0
    total_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        #print(inputs.shape)
        #print(inputs.shape)  
        # forward + backward + optimize
        #print(inputs[0].shape)
        outputs,f = net(inputs)
        #print(outputs.shape)
        #print(labels.shape)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        if(loss.item() > 1000):
            print(loss.item())
            for param in net.parameters():
                print(param.data)
        # print statistics
        running_loss += loss.item()
        total_loss += loss.item()
        if i % printby == printby-1:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.5f' %
                  (epoch + 1, i + 1, running_loss / printby))
            running_loss = 0.0
    return total_loss/(i+1)
def acc(net, classes):
    len_c = len(classes)
    class_correct = list(0. for i in range(len_c+1))
    class_total = list(0. for i in range(len_c+1))
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images = images.cuda()
            labels = labels.cuda()
            outputs,_ = net(images)
            _, predicted = torch.max(outputs, 1)
            c = (predicted == labels).squeeze()
            for i in range(4):
                label = labels[i]
                class_correct[label] += c[i].item()
                class_correct[len_c] += c[i].item()
                class_total[label] += 1
                class_total[len_c] += 1
                
    cl = classes + ["total"]
    print(class_total)
    for i in range(len_c+1):
        print('Accuracy of %5s : %2.2f %%' % (
            cl[i], 100 * class_correct[i] / class_total[i]))      
        
       

In [14]:
#when before bias=False

criterion = nn.CrossEntropyLoss().cuda()
optimizer = optim.Adam(net.parameters(),lr=0.0000001)

for epoch in range(100):  # loop over the dataset multiple times
    train(net, optimizer, criterion)
print('Finished Training')


[1,    50] loss: 0.08704
[1,   100] loss: 0.07618
[1,   150] loss: 0.09417
[2,    50] loss: 0.07522
[2,   100] loss: 0.07641
[2,   150] loss: 0.06883
[3,    50] loss: 0.05741
[3,   100] loss: 0.05197
[3,   150] loss: 0.06225
[4,    50] loss: 0.06855
[4,   100] loss: 0.07639
[4,   150] loss: 0.07025
[5,    50] loss: 0.05814
[5,   100] loss: 0.05310
[5,   150] loss: 0.07236
[6,    50] loss: 0.04998
[6,   100] loss: 0.05608
[6,   150] loss: 0.05480
[7,    50] loss: 0.05781
[7,   100] loss: 0.05216
[7,   150] loss: 0.05645
[8,    50] loss: 0.05173
[8,   100] loss: 0.04635
[8,   150] loss: 0.05850
[9,    50] loss: 0.03757
[9,   100] loss: 0.06551
[9,   150] loss: 0.04302
[10,    50] loss: 0.05796
[10,   100] loss: 0.05595
[10,   150] loss: 0.05308
[11,    50] loss: 0.03705
[11,   100] loss: 0.06485
[11,   150] loss: 0.03586
[12,    50] loss: 0.05285
[12,   100] loss: 0.05121
[12,   150] loss: 0.04786
[13,    50] loss: 0.03179
[13,   100] loss: 0.04239
[13,   150] loss: 0.04869
[14,    50] l

In [30]:
#when before bias=False


classes =  ('airplance', 'bird', 'car', 'cat', 'deer', 'dog', 'horse', 'monkey', 'ship', 'truck')
acc(net, classes)

Accuracy of airplance : 72.37 %
Accuracy of  bird : 51.55 %
Accuracy of   car : 55.66 %
Accuracy of   cat : 35.24 %
Accuracy of  deer : 47.92 %
Accuracy of   dog : 40.87 %
Accuracy of horse : 63.71 %
Accuracy of monkey : 50.54 %
Accuracy of  ship : 65.69 %
Accuracy of truck : 47.67 %


In [8]:
#when before random crop

lr = 0.001
for i in range(0, 30):
    if(i%5 == 0):
        lr = lr*0.1
    criterion = nn.CrossEntropyLoss().cuda()
    optimizer = optim.Adam(net.parameters(),lr=lr)

    for epoch in range(10):  # loop over the dataset multiple times
        train(net, optimizer, criterion)
    print('Finished Training')
    
    acc(net, classes)

[1,    50] loss: 2.19843
[1,   100] loss: 1.96804
[1,   150] loss: 1.89489
[2,    50] loss: 1.77191
[2,   100] loss: 1.71180
[2,   150] loss: 1.72148
[3,    50] loss: 1.58431
[3,   100] loss: 1.56137
[3,   150] loss: 1.54256
[4,    50] loss: 1.48808
[4,   100] loss: 1.50431
[4,   150] loss: 1.44775
[5,    50] loss: 1.36974
[5,   100] loss: 1.38364
[5,   150] loss: 1.39414
[6,    50] loss: 1.27901
[6,   100] loss: 1.26872
[6,   150] loss: 1.30427
[7,    50] loss: 1.18350
[7,   100] loss: 1.19454
[7,   150] loss: 1.16670
[8,    50] loss: 1.08393
[8,   100] loss: 1.09977
[8,   150] loss: 1.13437
[9,    50] loss: 0.91816
[9,   100] loss: 1.02358
[9,   150] loss: 1.01626
[10,    50] loss: 0.89448
[10,   100] loss: 0.90049
[10,   150] loss: 0.94486
Finished Training
Accuracy of airplance : 72.37 %
Accuracy of  bird : 56.70 %
Accuracy of   car : 52.83 %
Accuracy of   cat : 32.38 %
Accuracy of  deer : 44.79 %
Accuracy of   dog : 12.17 %
Accuracy of horse : 62.90 %
Accuracy of monkey : 65.59 %


[1,    50] loss: 0.00014
[1,   100] loss: 0.00024
[1,   150] loss: 0.00015
[2,    50] loss: 0.00022
[2,   100] loss: 0.00017
[2,   150] loss: 0.00017
[3,    50] loss: 0.00011
[3,   100] loss: 0.00019
[3,   150] loss: 0.00026
[4,    50] loss: 0.00156
[4,   100] loss: 0.00073
[4,   150] loss: 0.00063
[5,    50] loss: 0.00029
[5,   100] loss: 0.00036
[5,   150] loss: 0.00170
[6,    50] loss: 0.00309
[6,   100] loss: 0.00175
[6,   150] loss: 0.00155
[7,    50] loss: 0.00064
[7,   100] loss: 0.00025
[7,   150] loss: 0.00037
[8,    50] loss: 0.00058
[8,   100] loss: 0.00044
[8,   150] loss: 0.00017
[9,    50] loss: 0.00024
[9,   100] loss: 0.00019
[9,   150] loss: 0.00020
[10,    50] loss: 0.00014
[10,   100] loss: 0.00020
[10,   150] loss: 0.00022
Finished Training
Accuracy of airplance : 78.95 %
Accuracy of  bird : 69.07 %
Accuracy of   car : 81.13 %
Accuracy of   cat : 40.00 %
Accuracy of  deer : 65.62 %
Accuracy of   dog : 44.35 %
Accuracy of horse : 69.35 %
Accuracy of monkey : 66.67 %


[1,    50] loss: 0.00003
[1,   100] loss: 0.00005
[1,   150] loss: 0.00003
[2,    50] loss: 0.00002
[2,   100] loss: 0.00003
[2,   150] loss: 0.00004
[3,    50] loss: 0.00003
[3,   100] loss: 0.00015
[3,   150] loss: 0.00004
[4,    50] loss: 0.00002
[4,   100] loss: 0.00003
[4,   150] loss: 0.00003
[5,    50] loss: 0.00002
[5,   100] loss: 0.00002
[5,   150] loss: 0.00007
[6,    50] loss: 0.00004
[6,   100] loss: 0.00003
[6,   150] loss: 0.00003
[7,    50] loss: 0.00004
[7,   100] loss: 0.00002
[7,   150] loss: 0.00003
[8,    50] loss: 0.00003
[8,   100] loss: 0.00002
[8,   150] loss: 0.00008
[9,    50] loss: 0.00005
[9,   100] loss: 0.00003
[9,   150] loss: 0.00002
[10,    50] loss: 0.00003
[10,   100] loss: 0.00005
[10,   150] loss: 0.00006
Finished Training
Accuracy of airplance : 80.26 %
Accuracy of  bird : 72.16 %
Accuracy of   car : 83.96 %
Accuracy of   cat : 39.05 %
Accuracy of  deer : 66.67 %
Accuracy of   dog : 39.13 %
Accuracy of horse : 70.97 %
Accuracy of monkey : 65.59 %


[1,    50] loss: 0.00002
[1,   100] loss: 0.00002
[1,   150] loss: 0.00004
[2,    50] loss: 0.00003
[2,   100] loss: 0.00003
[2,   150] loss: 0.00003
[3,    50] loss: 0.00009
[3,   100] loss: 0.00002
[3,   150] loss: 0.00003
[4,    50] loss: 0.00016
[4,   100] loss: 0.00003
[4,   150] loss: 0.00005
[5,    50] loss: 0.00002
[5,   100] loss: 0.00004
[5,   150] loss: 0.00010
[6,    50] loss: 0.00003
[6,   100] loss: 0.00006
[6,   150] loss: 0.00002
[7,    50] loss: 0.00003
[7,   100] loss: 0.00002
[7,   150] loss: 0.00003
[8,    50] loss: 0.00002
[8,   100] loss: 0.00004
[8,   150] loss: 0.00004
[9,    50] loss: 0.00002
[9,   100] loss: 0.00005
[9,   150] loss: 0.00004
[10,    50] loss: 0.00002
[10,   100] loss: 0.00003
[10,   150] loss: 0.00009
Finished Training
Accuracy of airplance : 78.95 %
Accuracy of  bird : 72.16 %
Accuracy of   car : 83.96 %
Accuracy of   cat : 39.05 %
Accuracy of  deer : 66.67 %
Accuracy of   dog : 39.13 %
Accuracy of horse : 70.97 %
Accuracy of monkey : 65.59 %


In [9]:
#when after random crop
#In this case, we find that the initial learning rate of 0.1 is slightly too large to start converging
lr = 0.1
epoch = 0
for i in range(0, 30):
    if(i%3 == 0):
        lr = lr*0.1
        torch.save(net, str(epoch)+"ResNet.pt")
        print(lr)
    criterion = nn.CrossEntropyLoss().cuda()
    optimizer = optim.Adam(net.parameters(),lr=lr)

    for j in range(10):  # loop over the dataset multiple times
        train(net, optimizer, criterion)
        epoch += 1
    print('Finished Training')
    
    acc(net, classes)

/lvmdata/ees/anaconda3/envs/tensorflow35/lib/python3.5/site-packages/torch/serialization.py:241: UserWarning: Couldn't retrieve source code for container of type ResNet. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/lvmdata/ees/anaconda3/envs/tensorflow35/lib/python3.5/site-packages/torch/serialization.py:241: UserWarning: Couldn't retrieve source code for container of type BottleNeck. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


0.010000000000000002
[1,    50] loss: 2.80422
[1,   100] loss: 2.37631
[1,   150] loss: 2.18192
[2,    50] loss: 2.01642
[2,   100] loss: 1.98285
[2,   150] loss: 1.92821
[3,    50] loss: 1.91784
[3,   100] loss: 1.83816
[3,   150] loss: 1.79727
[4,    50] loss: 1.75790
[4,   100] loss: 1.80242
[4,   150] loss: 1.72343
[5,    50] loss: 1.79246
[5,   100] loss: 1.72310
[5,   150] loss: 1.72773
[6,    50] loss: 1.70475
[6,   100] loss: 1.68635
[6,   150] loss: 1.66438
[7,    50] loss: 1.65835
[7,   100] loss: 1.67756
[7,   150] loss: 1.65095
[8,    50] loss: 1.65986
[8,   100] loss: 1.65531
[8,   150] loss: 1.63793
[9,    50] loss: 1.61612
[9,   100] loss: 1.66719
[9,   150] loss: 1.58352
[10,    50] loss: 1.60465
[10,   100] loss: 1.56013
[10,   150] loss: 1.60562
Finished Training
Accuracy of airplance : 57.89 %
Accuracy of  bird : 38.14 %
Accuracy of   car : 68.87 %
Accuracy of   cat : 9.52 %
Accuracy of  deer : 32.29 %
Accuracy of   dog : 4.35 %
Accuracy of horse : 43.55 %
Accuracy o

[78,   150] loss: 0.66254
[79,    50] loss: 0.67594
[79,   100] loss: 0.66598
[79,   150] loss: 0.71030
[80,    50] loss: 0.66610
[80,   100] loss: 0.65912
[80,   150] loss: 0.67140
Finished Training
Accuracy of airplance : 81.58 %
Accuracy of  bird : 63.92 %
Accuracy of   car : 78.30 %
Accuracy of   cat : 51.43 %
Accuracy of  deer : 66.67 %
Accuracy of   dog : 47.83 %
Accuracy of horse : 69.35 %
Accuracy of monkey : 65.59 %
Accuracy of  ship : 90.20 %
Accuracy of truck : 76.74 %
[81,    50] loss: 0.66045
[81,   100] loss: 0.69019
[81,   150] loss: 0.63858
[82,    50] loss: 0.64554
[82,   100] loss: 0.65756
[82,   150] loss: 0.72191
[83,    50] loss: 0.64697
[83,   100] loss: 0.64163
[83,   150] loss: 0.68119
[84,    50] loss: 0.64703
[84,   100] loss: 0.65601
[84,   150] loss: 0.67563
[85,    50] loss: 0.62927
[85,   100] loss: 0.67042
[85,   150] loss: 0.65948
[86,    50] loss: 0.67667
[86,   100] loss: 0.62257
[86,   150] loss: 0.68254
[87,    50] loss: 0.67870
[87,   100] loss: 0.6

1.0000000000000005e-07
[151,    50] loss: 0.66561
[151,   100] loss: 0.62281
[151,   150] loss: 0.66306
[152,    50] loss: 0.63543
[152,   100] loss: 0.63948
[152,   150] loss: 0.66622
[153,    50] loss: 0.66217
[153,   100] loss: 0.64582
[153,   150] loss: 0.65008
[154,    50] loss: 0.63096
[154,   100] loss: 0.69028
[154,   150] loss: 0.67587
[155,    50] loss: 0.67727
[155,   100] loss: 0.65454
[155,   150] loss: 0.62892
[156,    50] loss: 0.65664
[156,   100] loss: 0.65060
[156,   150] loss: 0.64475
[157,    50] loss: 0.67873
[157,   100] loss: 0.62635
[157,   150] loss: 0.64886
[158,    50] loss: 0.65106
[158,   100] loss: 0.64621
[158,   150] loss: 0.66157
[159,    50] loss: 0.65478
[159,   100] loss: 0.68053
[159,   150] loss: 0.67697
[160,    50] loss: 0.64020
[160,   100] loss: 0.65271
[160,   150] loss: 0.65595
Finished Training
Accuracy of airplance : 80.26 %
Accuracy of  bird : 69.07 %
Accuracy of   car : 83.96 %
Accuracy of   cat : 50.48 %
Accuracy of  deer : 63.54 %
Accur

[225,   100] loss: 0.63253
[225,   150] loss: 0.64186
[226,    50] loss: 0.65263
[226,   100] loss: 0.64865
[226,   150] loss: 0.62670
[227,    50] loss: 0.66477
[227,   100] loss: 0.66369
[227,   150] loss: 0.63793
[228,    50] loss: 0.60811
[228,   100] loss: 0.66775
[228,   150] loss: 0.65317
[229,    50] loss: 0.66000
[229,   100] loss: 0.68952
[229,   150] loss: 0.63897
[230,    50] loss: 0.66337
[230,   100] loss: 0.62489
[230,   150] loss: 0.62889
Finished Training
Accuracy of airplance : 80.26 %
Accuracy of  bird : 64.95 %
Accuracy of   car : 82.08 %
Accuracy of   cat : 48.57 %
Accuracy of  deer : 66.67 %
Accuracy of   dog : 40.87 %
Accuracy of horse : 76.61 %
Accuracy of monkey : 68.82 %
Accuracy of  ship : 90.20 %
Accuracy of truck : 76.74 %
[231,    50] loss: 0.65423
[231,   100] loss: 0.65064
[231,   150] loss: 0.62929
[232,    50] loss: 0.67922
[232,   100] loss: 0.63938
[232,   150] loss: 0.63257
[233,    50] loss: 0.64095
[233,   100] loss: 0.67345
[233,   150] loss: 0.6

[300,    50] loss: 0.66605
[300,   100] loss: 0.65308
[300,   150] loss: 0.61566
Finished Training
Accuracy of airplance : 80.26 %
Accuracy of  bird : 63.92 %
Accuracy of   car : 74.53 %
Accuracy of   cat : 52.38 %
Accuracy of  deer : 63.54 %
Accuracy of   dog : 43.48 %
Accuracy of horse : 70.97 %
Accuracy of monkey : 68.82 %
Accuracy of  ship : 86.27 %
Accuracy of truck : 74.42 %


In [14]:
classes =  ('airplance', 'bird', 'car', 'cat', 'deer', 'dog', 'horse', 'monkey', 'ship', 'truck')
acc(net, classes)

Accuracy of airplance : 78.95 %
Accuracy of  bird : 65.98 %
Accuracy of   car : 77.36 %
Accuracy of   cat : 49.52 %
Accuracy of  deer : 71.88 %
Accuracy of   dog : 43.48 %
Accuracy of horse : 70.97 %
Accuracy of monkey : 72.04 %
Accuracy of  ship : 89.22 %
Accuracy of truck : 74.42 %
Accuracy of total : 68.70 %


In [15]:
lr = 0.000000000000001
if(i%3 == 0):
    lr = lr*0.1
    torch.save(net, str(epoch)+"ResNet.pt")
    print(lr)
criterion = nn.CrossEntropyLoss().cuda()
optimizer = optim.Adam(net.parameters(),lr=lr)

for j in range(10):  # loop over the dataset multiple times
    train(net, optimizer, criterion)
    epoch += 1
print('Finished Training')

acc(net, classes)

[301,    50] loss: 0.62088
[301,   100] loss: 0.65456
[301,   150] loss: 0.63559
[302,    50] loss: 0.65563
[302,   100] loss: 0.65762
[302,   150] loss: 0.63489
[303,    50] loss: 0.65556
[303,   100] loss: 0.61902
[303,   150] loss: 0.64065
[304,    50] loss: 0.65469
[304,   100] loss: 0.67801
[304,   150] loss: 0.66140
[305,    50] loss: 0.68515
[305,   100] loss: 0.62532
[305,   150] loss: 0.66133
[306,    50] loss: 0.64707
[306,   100] loss: 0.67406
[306,   150] loss: 0.62302
[307,    50] loss: 0.62893
[307,   100] loss: 0.61117
[307,   150] loss: 0.70224
[308,    50] loss: 0.65163
[308,   100] loss: 0.67001
[308,   150] loss: 0.64692
[309,    50] loss: 0.66664
[309,   100] loss: 0.62562
[309,   150] loss: 0.63770
[310,    50] loss: 0.63068
[310,   100] loss: 0.65397
[310,   150] loss: 0.64158
Finished Training
Accuracy of airplance : 76.32 %
Accuracy of  bird : 68.04 %
Accuracy of   car : 80.19 %
Accuracy of   cat : 47.62 %
Accuracy of  deer : 66.67 %
Accuracy of   dog : 33.91 %


ZeroDivisionError: float division by zero

In [28]:
lr = 0.00000000000000001
if(i%3 == 0):
    lr = lr*0.1
    torch.save(net, str(epoch)+"ResNet.pt")
    print(lr)
criterion = nn.CrossEntropyLoss().cuda()
optimizer = optim.Adam(net.parameters(),lr=lr)

for j in range(10):  # loop over the dataset multiple times
    train(net, optimizer, criterion)
    epoch += 1
print('Finished Training')

acc(net, classes)

[311,    50] loss: 0.63372
[311,   100] loss: 0.64323
[311,   150] loss: 0.65851
[312,    50] loss: 0.65423
[312,   100] loss: 0.62969
[312,   150] loss: 0.69501
[313,    50] loss: 0.70182
[313,   100] loss: 0.61526
[313,   150] loss: 0.62426
[314,    50] loss: 0.65496
[314,   100] loss: 0.66951
[314,   150] loss: 0.65212
[315,    50] loss: 0.62231
[315,   100] loss: 0.66094
[315,   150] loss: 0.65354
[316,    50] loss: 0.62570
[316,   100] loss: 0.65484
[316,   150] loss: 0.65840
[317,    50] loss: 0.63019
[317,   100] loss: 0.69004
[317,   150] loss: 0.66735
[318,    50] loss: 0.68197
[318,   100] loss: 0.65788
[318,   150] loss: 0.65220
[319,    50] loss: 0.63458
[319,   100] loss: 0.62570
[319,   150] loss: 0.67097
[320,    50] loss: 0.69647
[320,   100] loss: 0.64579
[320,   150] loss: 0.67064
Finished Training
[76.0, 97.0, 106.0, 105.0, 96.0, 115.0, 124.0, 93.0, 102.0, 86.0, 1000.0]
Accuracy of airplance : 78.95 %
Accuracy of  bird : 63.92 %
Accuracy of   car : 82.08 %
Accuracy o

In [31]:
#after remove random in test
acc(net, classes)

[76.0, 97.0, 106.0, 105.0, 96.0, 115.0, 124.0, 93.0, 102.0, 86.0, 1000.0]
Accuracy of airplance : 82.89 %
Accuracy of  bird : 60.82 %
Accuracy of   car : 83.96 %
Accuracy of   cat : 54.29 %
Accuracy of  deer : 68.75 %
Accuracy of   dog : 41.74 %
Accuracy of horse : 75.00 %
Accuracy of monkey : 66.67 %
Accuracy of  ship : 84.31 %
Accuracy of truck : 73.26 %
Accuracy of total : 68.60 %
